## Partie 1 Scrapping des données de bars de Toulouse

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
urls = ["https://www.schlouk-map.com/fr/cities/toulouse/happy-hour", "https://www.schlouk-map.com/fr/cities/toulouse/happy-hour?page=2"]

In [3]:
bar_liens = []

for url in urls:# Recuperer les liens des bars
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    for card in soup.find_all('div', 'card-body'):
        bar_liens.append("https://www.schlouk-map.com"+card.find('a').get('href'))

In [4]:
bar_lien = bar_liens[0]

In [5]:
r = requests.get(bar_lien)
soup = BeautifulSoup(r.text)

In [6]:
nom = soup.find('h1','mb-0 d-inline-block').get_text().replace('\n','')
nom

'Matabiau Décapsule Club'

In [8]:
# Recuperation de la table horaire pour les infos 
table = soup.find('table', 'table table-sm table-borderless mb-0')
horraires = []
for l in table.find_all('tr'):
    jour_name = l.find('td').get_text().replace('\n','')
    heure = l.find('td','float-right float-md-none').get_text().replace('\n','')
    jour = {
        'jour': jour_name,
        'heure' : heure
    }
    horraires.append(jour)

print(horraires)

[{'jour': 'Lundi', 'heure': '15:30 - 20:00'}, {'jour': 'Mardi', 'heure': '13:00 - 23:00'}, {'jour': 'Mercredi', 'heure': '13:00 - 23:00'}, {'jour': 'Jeudi', 'heure': '13:00 - 23:00'}, {'jour': 'Vendredi', 'heure': '13:00 - 23:00'}, {'jour': 'Samedi', 'heure': '13:00 - 23:00'}, {'jour': 'Dimanche', 'heure': 'Fermé'}]


In [9]:
#Recuperation de la table tarifs et infos 
tarif_table = soup.find('table','table table-sm table-borderless table-drinks')

for l in tarif_table.find_all('tr')[1:]:
    
    boisson = l.find('td','text-truncate max-text-width-px').get_text().replace('\n','')
    tarif_h = l.find_all('td','font-weight-bold text-right')[0].get_text().replace('\n','').replace('\xa0','')
    tarif_s = l.find_all('td','font-weight-bold text-right')[1].get_text().replace('\n','').replace('\xa0','')
    prix = {
            "boisson" : boisson,
            "prix_happy_hour" : tarif_h,
            "prix_standard" : tarif_s
        }
    print(prix)
    

{'boisson': 'Bière blonde(50cl)', 'prix_happy_hour': '3,70€', 'prix_standard': '4,90€'}
{'boisson': 'Bière blonde(25cl)', 'prix_happy_hour': '- ', 'prix_standard': '2,80€'}


In [10]:
lig = tarif_table.find_all('tr')[1]

In [13]:
#Recuperation de la table service et des infos 

service_table = soup.find('table','table table-sm table-borderless table-services mb-0')
services = []
for l in service_table.find_all('tr'):
    try:
        condition = l.find('span','font-weight-bold text-nowrap').get_text()
    except:
        condition = 'Oui'

    if condition != 'Non':
        service = l.find('td','text-truncate').get_text()
        services.append(service)

print(services)


[' Restauration\n', ' Vente à emporter\n', ' Wifi gratuit\n', '🌬 Air climatisé\n', ' Fléchettes\n', ' Jeux de société\n']


In [15]:
#Creation d'une fonction pour boucler sur tous les bars et recup les infos
def scrap_shlouk(bar_liens):
    
    data = []
    for lien in bar_liens:
        print(lien)
        r = requests.get(lien)
        soup = BeautifulSoup(r.text)

        nom = soup.find('h1','mb-0 d-inline-block').get_text().replace('\n','')

        table = soup.find('table', 'table table-sm table-borderless mb-0')
        horraires = []
        for l in table.find_all('tr'):
            jour_name = l.find('td').get_text().replace('\n','')
            heure = l.find('td','float-right float-md-none').get_text().replace('\n','')
            jour = {
                'jour': jour_name,
                'heure' : heure
            }
            horraires.append(jour)

        tarif_table = soup.find('table','table table-sm table-borderless table-drinks')

        tarifs = []

        try :
            for l in tarif_table.find_all('tr')[1:]:
            
                boisson = l.find('td','text-truncate max-text-width-px').get_text().replace('\n','')
                tarif_h = l.find_all('td','font-weight-bold text-right')[0].get_text().replace('\n','').replace('\xa0','')
                tarif_s = l.find_all('td','font-weight-bold text-right')[1].get_text().replace('\n','').replace('\xa0','')
                prix = {
                        "boisson" : boisson,
                        "prix_happy_hour" : tarif_h,
                        "prix_standard" : tarif_s
                    }
                tarifs.append(prix)
        except:
            tarifs.append('No data available')
            
        service_table = soup.find('table','table table-sm table-borderless table-services mb-0')
        services = []
        for l in service_table.find_all('tr'):
            try:
                condition = l.find('span','font-weight-bold text-success text-nowrap').get_text()
            except:
                condition = 'Non'

            if condition == 'Oui':
                service = l.find('td','text-truncate').get_text()
                services.append(service)


        bar = {
            'nom' : nom,
            'horraires' : horraires,
            'tarifs' : tarifs,
            'services' : services,
            'lien' : lien

        }
    
        data.append(bar)

    return data    

In [17]:
#Verification des liens recupérers et de la taille de data qui doit être 67 
data = scrap_shlouk(bar_liens)

len(data)

https://www.schlouk-map.com/fr/places/matabiau-decapsule-club
https://www.schlouk-map.com/fr/places/tom-pouce
https://www.schlouk-map.com/fr/places/the-black-lion-1
https://www.schlouk-map.com/fr/places/barricot
https://www.schlouk-map.com/fr/places/la-maison-1
https://www.schlouk-map.com/fr/places/el-circo
https://www.schlouk-map.com/fr/places/o-boudu-pont
https://www.schlouk-map.com/fr/places/le-tchin
https://www.schlouk-map.com/fr/places/the-botanist-pub
https://www.schlouk-map.com/fr/places/le-pery
https://www.schlouk-map.com/fr/places/le-biergarten
https://www.schlouk-map.com/fr/places/cafe-oz-2
https://www.schlouk-map.com/fr/places/the-frog-rosbif-2
https://www.schlouk-map.com/fr/places/the-petit-london
https://www.schlouk-map.com/fr/places/the-danu
https://www.schlouk-map.com/fr/places/snapper-rock
https://www.schlouk-map.com/fr/places/the-hopscotch
https://www.schlouk-map.com/fr/places/le-bistrologue
https://www.schlouk-map.com/fr/places/1862-artybar
https://www.schlouk-map.com

67

In [19]:
#Verif des données récup pour un bar 
#PS : les Aaaaa sont ceux qui y a indiqués dans les boissons pour ce bar cf voir le site et ce bar 
data[56]

{'nom': 'Le Biergarten Toulouse',
 'horraires': [{'jour': 'Lundi', 'heure': '17:00 - 01:00'},
  {'jour': 'Mardi', 'heure': '17:00 - 02:00'},
  {'jour': 'Mercredi', 'heure': '17:00 - 02:00'},
  {'jour': 'Jeudi', 'heure': '17:00 - 02:00'},
  {'jour': 'Vendredi', 'heure': '17:00 - 02:00'},
  {'jour': 'Samedi', 'heure': '17:00 - 02:00'},
  {'jour': 'Dimanche', 'heure': '17:00 - 00:00'}],
 'tarifs': [{'boisson': 'Oktoberfest Bier(50cl)',
   'prix_happy_hour': '- ',
   'prix_standard': '7,00€'},
  {'boisson': 'Munich(50cl)',
   'prix_happy_hour': '- ',
   'prix_standard': '6,00€'},
  {'boisson': ' Dunkel(50cl)',
   'prix_happy_hour': '- ',
   'prix_standard': '7,50€'},
  {'boisson': ' Saint Thomas(50cl)',
   'prix_happy_hour': '- ',
   'prix_standard': '8,00€'},
  {'boisson': 'aaaaaaaaaaaaaaaaaa ',
   'prix_happy_hour': '3,00€',
   'prix_standard': '1,00€'},
  {'boisson': 'aaaaaaaaaaa ',
   'prix_happy_hour': '2,00€',
   'prix_standard': '6,00€'},
  {'boisson': 'aaaaaaaaaaa ',
   'prix_happy

## Partie 2 : Creation du DF et analyse meilleur deal et prix moyen

In [20]:
import pandas as pd

In [21]:
df = pd.DataFrame(data)


In [24]:
tarifs_df = pd.concat([pd.DataFrame(x) for x in df['tarifs']], keys=df['nom']).reset_index(level=1, drop=True)

tarifs_df

,boisson,prix_happy_hour,prix_standard,0
nom,,,,
Matabiau Décapsule Club,Bière blonde(50cl),"3,70€","4,90€",NaN
Matabiau Décapsule Club,Bière blonde(25cl),-,"2,80€",NaN
Tom Pouce,1664 Blanche(50cl),"5,50€","7,00€",NaN
The Black Lion,Kronenbourg(50cl),"4,00€",-,NaN
The Black Lion,Carlsberg(50cl),"4,50€",-,NaN
...,...,...,...,...
Le Filochard,aaaaaaaaaaaaaaaaaaa,"3,00€","8,00€",NaN
Le Filochard,aaaaaaaaaaaaaaaaaaaa,"8,00€","7,00€",NaN
Le Filochard,aaaaaaaaaaaa,"3,00€","6,00€",NaN


In [25]:
#Creation fonction pour meilleur deal 
def calculate_deal(row):
    try:
        eco = row["prix_standard"] - row["prix_happy_hour"]
    except:
        eco = 0
    
    return eco


In [26]:
#Fonction nettoyage 
def clean_price_hp(row):
    try:
        prix = row["prix_happy_hour"].split("€")[0]
        prix = prix.replace(',', '.')
        prix = float(prix)
    except:
        prix = 0
    return prix

def clean_price_s(row):
    try:
        prix = row["prix_standard"].split("€")[0]
        prix = prix.replace(',', '.')
        prix = float(prix)
    except:
        prix = 0
    return prix

In [27]:
#Application des fonnctions nettoyage sur les colonnes 
tarifs_df["prix_happy_hour"] = tarifs_df.apply(clean_price_hp,axis=1)
tarifs_df["prix_standard"] = tarifs_df.apply(clean_price_s,axis=1)

In [31]:
tarifs_df = tarifs_df.groupby('nom').agg({'prix_happy_hour': 'mean', 'prix_standard': 'mean'})

tarifs_df

,prix_happy_hour,prix_standard
nom,,
1862 Artybar,6.500000,3.000000
Au Fût et à Mesure,4.307692,3.692308
Au Poêle de la Bête,2.000000,0.000000
Bar Des Zés,3.576923,4.423077
Bar Martini Club,2.500000,0.000000
...,...,...
The Petit London,5.346154,4.076923
The Tower of London,5.515385,2.076923
Tom Pouce,5.500000,7.000000


In [45]:
prix_happy_hours_non_zero = tarifs_df[tarifs_df['prix_happy_hour'] != 0]['prix_happy_hour']
moyenne_prix_happy_hours = prix_happy_hours_non_zero.mean().round(2)

print(moyenne_prix_happy_hours)

prix_s_non_zero = tarifs_df[tarifs_df['prix_standard'] != 0]['prix_standard']
moyenne_prix_s = prix_s_non_zero.mean().round(2)

print(moyenne_prix_s)

print("Les prix moyens en Happy hours des bars de Toulouse sont : 4.23€" )
print("Les prix moyens en standard des bars de Toulouse sont : 4.48€" )

4.23
4.48
Les prix moyens en Happy hours des bars de Toulouse sont : 2;84e
Les prix moyens en standard des bars de Toulouse sont : 2;84e


In [32]:
#Creation de la colonne eco pour meilleur deal 
tarifs_df["eco"] = tarifs_df.apply(calculate_deal,axis=1)
deals = tarifs_df[tarifs_df["prix_happy_hour"]!=0]

In [33]:
deals = deals.sort_values(by='eco', ascending=False)
top5 = deals.head(5)

In [213]:
top5.to_csv('top5.csv')

In [46]:
import smtplib
import email.mime
import mailtrap as mt

In [47]:
from jinja2 import Environment, FileSystemLoader

# Charger le répertoire contenant les templates
env = Environment(loader=FileSystemLoader('/Users/HP/Documents/SCRAPPING/HappyHourProject/templates'))

env

In [48]:
# Charger le template HTML
template = env.get_template('report_template.html')
template

<Template 'report_template.html'>

In [51]:
bars_l = [
    {'name': 'Le Pery', 'address': '22 Rue Gabriel Péri, 31000 Toulouse', 'happy_hour_price': 2.54, 'normal_price': 6.57, 'savings': 4.03},
    {'name': 'Le Biergarten', 'address': '60 Grand-Rue Saint-Michel, 31400 Toulouse', 'happy_hour_price': 2.92, 'normal_price': 6.07, 'savings': 3.15},
    {'name': 'Cacahuète', 'address': '37 Rue Peyrolières, 31000 Toulouse', 'happy_hour_price': 2.15, 'normal_price': 5.21, 'savings': 3.06},
    {'name': 'MAD (Meet And Drink)', 'address': '4 Allées Charles de Fitte, 31300 Toulouse', 'happy_hour_price': 3.0, 'normal_price': 5.0, 'savings': 2.0},
    {'name': 'Barricot', 'address': '8 Esplanade Compans Caffarelli, 31000 Toulouse', 'happy_hour_price': 2.15, 'normal_price': 5.01, 'savings': 2.86}
]


# Rendre le template avec les données
html_output = template.render(bars_l=bars_l)

# Écrire le résultat dans un fichier HTML
with open('/Users/HP/Documents/SCRAPPING/HappyHourProject/templates/report_template.html', 'w') as f:
    f.write(html_output)

In [52]:
smtpObj = smtplib.SMTP

In [53]:

mail = mt.Mail(
    sender=mt.Address(email="mailtrap@demomailtrap.com", name="Mailtrap Test"),
    to=[mt.Address(email="damien.lauger.edu@groupe-gema.com", name="Recipient Name")],
    subject="Top 5 bars Toulouse en Happy Hour",
    text="voici un recap des meilleur deal en happy hour à toulouse",
    html="""
    <!DOCTYPE html>
<html lang="fr">
<head>
    <meta charset="UTF-8">
    <title>Rapport des Happy Hours à Toulouse</title>
    <style>
        body { font-family: Arial, sans-serif; }
        table { width: 100%; border-collapse: collapse; }
        th, td { border: 1px solid #ddd; padding: 8px; text-align: left; }
        th { background-color: #f2f2f2; }
    </style>
</head>
<body>
    <h1>Rapport des Happy Hours à Toulouse</h1>
    <h2>Top 5 des Meilleurs Deals</h2>
    <table>
        <tr>
            <th>Nom du Bar</th>
            <th>Adresse</th>
            <th>Prix Happy Hour</th>
            <th>Prix Normal</th>
            <th>Économie</th>
        </tr>
        
        <tr>
            <td>Le Pery</td>
            <td>22 Rue Gabriel Péri, 31000 Toulouse</td>
            <td>2.54 €</td>
            <td>6.57 €</td>
            <td>4.03€</td>
        </tr>
        
        <tr>
            <td>Le Biergarten</td>
            <td>60 Grand-Rue Saint-Michel, 31400 Toulouse</td>
            <td>2.92 €</td>
            <td>6.07 €</td>
            <td>3.15€</td>
        </tr>
        
        <tr>
            <td>Cachauète</td>
            <td>37 Rue Peyrolières, 31000 Toulouse</td>
            <td>2.15 €</td>
            <td>5.21 €</td>
            <td>3.06 €</td>
        </tr>
        
        <tr>
            <td>MAD (Meet And Drink)</td>
            <td>4 Allées Charles de Fitte, 31300 Toulouse</td>
            <td>3.0 €</td>
            <td>5.0 €</td>
            <td>2.0 €</td>
        </tr>
        
        <tr>
            <td>Barricot</td>
            <td>8 Esplanade Compans Caffarelli, 31000 Toulouse</td>
            <td>2.15 €</td>
            <td>5.01 €</td>
            <td>2.86 €</td>
        </tr>
        
    </table>
</body>
</html>
    """,
    # You can categorize this email or add custom headers as needed
    category="HTML Email",
    headers={"X-Custom-Header": "Value"}
)

# Initialize the MailtrapClient with your API token
client = mt.MailtrapClient(token="a379f75bac968553e81616b6df4abf78")

# Send the email
client.send(mail)

print("HTML email sent successfully.")

HTML email sent successfully.


### Card 

In [ ]:
from geopy.geocoders import Nominatim
import folium
from geopy.exc import GeocoderUnavailable

In [231]:
geolocator = Nominatim(user_agent="my_geocoder")

toulouse_location = geolocator.geocode("Toulouse, France")

m = folium.Map(location=[toulouse_location.latitude, toulouse_location.longitude], zoom_start=14)

In [223]:
data_carte= {
    "Nom du bar": [
        "Matabiau Décapsule Club", "Tom Pouce", "The Black Lion", "Barricot", "La Maison", 
        "El Circo", "Ô Boudu Pont", "Le Tchin", "The Botanist Pub", "Le Péry", 
        "Le Biergarten", "Café Oz", "The Frog & Rosbif", "The Petit London", "The Danu", 
        "Snapper Rock", "The Hopscotch", "Le Bistrologue", "1862 Artybar", "Kraken Paradise", 
        "MAD (Meet and Drink)", "Le Delicatessen", "Bar Martini Club", "Borriquito Loco", 
        "L'Autruche", "Sauvage Social Pub", "Beer Mosaic", "L'Almanach Bistrot", 
        "Le Saint des Seins", "La Tireuse", "The London Town", "La Tantina De Burgos", 
        "Rooster & Beer", "Cacahuète", "Au Fût et à Mesure", "Café Populaire", 
        "The Tower of London", "La Cale Sèche", "G-Bar", "Au Poêle de la Bête", 
        "L'Evasion", "Le Comptoir", "Le Bibent", "El Chivito", "Mucho", "Dubliners", 
        "Moloko", "Taparcero", "Chez Tonton", "Wanted Jack Saloon", "Le Petit Voisin", 
        "La Goulue", "Smoking Barrel", "Le Bièrographe", "L Episode", "The Four Monkeys", 
        "Le Biergarten Toulouse", "L'Ancienne Belgique", "Bar Des Zés", "Le Carbet d'Oc", 
        "Pub O'Clock", "The George and Dragon", "The Melting Pot Pub", "Les Tricheurs", 
        "Baraka Jeux", "Dada", "Le Filochard"
    ],
    "Adresse": [
        "74 Rue Matabiau, 31000 Toulouse", "4 Place Roger Salengro, 31000 Toulouse", 
        "4 Allées Paul Feuga, 31000 Toulouse", "8 Esplanade Compans Caffarelli, 31000 Toulouse", 
        "9 Rue Gabriel Pérri, 31000 Toulouse", "61 Rue Pargaminières, 31000 Toulouse", 
        "1 Rue de la République, 31300 Toulouse", "22 Rue Saint-Bernard, 31000 Toulouse", 
        "33 Boulevard Maréchal Leclerc, 31000 Toulouse", "22 Rue Gabriel Péri, 31000 Toulouse", 
        "60 Grand-Rue Saint-Michel, 31400 Toulouse", "1 Rue Gabriel Péri, 31000 Toulouse", 
        "14 Rue de l'Industrie, 31000 Toulouse", "7 Rue Riquet, 31000 Toulouse", 
        "9 Rue du Pont Guilhemery, 31000 Toulouse", "4 Rue Gabriel Péri, 31000 Toulouse", 
        "3 Rue Bour Lormian, 31000 Toulouse", "2 Place du Ravelin, 31300 Toulouse", 
        "24 Rue Nicolas Bachelier, 31000 Toulouse", "34 Rue de la Colombette, 31000 Toulouse", 
        "4 Allées Charles de Fitte, 31300 Toulouse", "11 bis Rue Riquet, 31000 Toulouse", 
        "9 Rue d'Austerlitz, 31000 Toulouse", "25 Rue des Paradoux, 31000 Toulouse", 
        "1 Rue André Mercadier, 31000 Toulouse", "11 Place Etienne Esquirol, 31000 Toulouse", 
        "7bis Rue Pierre-Paul Riquet, 31000 Toulouse", "53 Rue de la République, 31300 Toulouse", 
        "5 Place Saint-Pierre, 31000 Toulouse", "24 Rue Pargaminières, 31000 Toulouse", 
        "14 Rue des Prêtres, 31000 Toulouse", "27 Avenue de la Garonnette, 31000 Toulouse", 
        "100 Rue Riquet, 31000 Toulouse", "21 Place de l'Estrapade, 31300 Toulouse", 
        "20 Rue Gabriel Péri, 31000 Toulouse", "9 Rue de la Colombette, 31000 Toulouse", 
        "39 Grande Rue Saint-Michel, 31400 Toulouse", "41 Rue Léon Gambetta, 31000 Toulouse", 
        "10 Rue du Pont Montaudran, 31000 Toulouse", "56 Rue d'Aubuisson, 31000 Toulouse", 
        "29 Grande Rue Saint-Michel, 31400 Toulouse", "12 Avenue de Grande Bretagne, 31300 Toulouse", 
        "5 Place du Capitole, 31000 Toulouse", "15 Rue des Blanchers, 31000 Toulouse", 
        "9 Place du Pont Neuf, 31000 Toulouse", "46 Avenue Marcel Langer, 31400 Toulouse", 
        "6 Rue Joutx Aigues, 31000 Toulouse", "63 Rue Pargaminières, 31000 Toulouse", 
        "16 Place Saint-Pierre, 31000 Toulouse", "2 Rue Raymond IV, 31000 Toulouse", 
        "37 Rue Peyrolières, 31000 Toulouse", "43 Boulevard Lascrosses, 31000 Toulouse", 
        "5 Place Jean Diebold, 31300 Toulouse", "12 Rue des Paradoux, 31000 Toulouse", 
        "3 Rue des Blanchers, 31000 Toulouse", "7 Rue de Metz, 31000 Toulouse", 
        "60 Grande Rue Saint-Michel, 31400 Toulouse", "16 Rue de la Trinité, 31000 Toulouse", 
        "49 Rue Pargaminières, 31000 Toulouse", "4 Rue des Filatiers, 31000 Toulouse", 
        "21 Boulevard de Strasbourg, 31000 Toulouse", "1 Place du Peyrou, 31000 Toulouse", 
        "26 Boulevard de Strasbourg, 31000 Toulouse", "34 Rue des Blanchers, 31000 Toulouse", 
        "1 Boulevard de la Gare, 31500 Toulouse", "27 Avenue Honoré Serres, 31000 Toulouse", 
        "8 Place du Pont Neuf, 31000 Toulouse"
    ]
}

df_map = pd.DataFrame(data_carte)
df_map

,Nom du bar,Adresse
0,Matabiau Décapsule Club,"74 Rue Matabiau, 31000 Toulouse"
1,Tom Pouce,"4 Place Roger Salengro, 31000 Toulouse"
2,The Black Lion,"4 Allées Paul Feuga, 31000 Toulouse"
3,Barricot,"8 Esplanade Compans Caffarelli, 31000 Toulouse"
4,La Maison,"9 Rue Gabriel Pérri, 31000 Toulouse"
...,...,...
62,The Melting Pot Pub,"26 Boulevard de Strasbourg, 31000 Toulouse"
63,Les Tricheurs,"34 Rue des Blanchers, 31000 Toulouse"
64,Baraka Jeux,"1 Boulevard de la Gare, 31500 Toulouse"
65,Dada,"27 Avenue Honoré Serres, 31000 Toulouse"


In [229]:
# Création de la carte centrée sur une position par défaut
m = folium.Map(location=[43.600000, 1.433333], zoom_start=12)

# Itération à travers les données du dataframe
for nom, adresse in zip(df_map['Nom du bar'], df_map['Adresse']):
    try:
        # Géocodage de l'adresse pour obtenir les coordonnées GPS
        location = geolocator.geocode(adresse)
        if location:
            # Création du texte pour le popup (nom du bar avec un lien et prix)
            popup_text = f'<strong>Nom:</strong> {nom}<br><strong>Adresse:</strong> {adresse}<br>'
            # Ajout d'un marqueur sur la carte avec les coordonnées GPS et le popup contenant le nom du bar et l'adresse
            folium.Marker([location.latitude, location.longitude], popup=popup_text,
                          tooltip=nom).add_to(m)
    except GeocoderUnavailable:
        print("Le service de géocodage est indisponible. Veuillez réessayer plus tard.")

# Sauvegarde de la carte sous forme de fichier HTML
m.save('map.html')


In [230]:
m